### MNIST

In [1]:
import torch.nn.functional as F
from torch_explain.nn.concepts import ConceptReasoningLayer, IntpLinearLayer1, IntpLinearLayer2, IntpLinearLayer3
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch_explain as te
from torch_explain import datasets
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

%matplotlib inline
import logging
import warnings
warnings.filterwarnings('ignore')
# Set the logging level for matplotlib to WARNING to suppress debug information
logging.getLogger('matplotlib').setLevel(logging.WARNING)


PATH = 'models/'

2024-07-23 19:37:02,101 - matplotlib - DEBUG - matplotlib data path: d:\Github\pytorch_explain\env\Lib\site-packages\matplotlib\mpl-data
2024-07-23 19:37:02,112 - matplotlib - DEBUG - CONFIGDIR=C:\Users\a_h9\.matplotlib
2024-07-23 19:37:02,160 - matplotlib - DEBUG - interactive is False
2024-07-23 19:37:02,164 - matplotlib - DEBUG - platform is win32
2024-07-23 19:37:02,324 - matplotlib - DEBUG - CACHEDIR=C:\Users\a_h9\.matplotlib
2024-07-23 19:37:02,337 - matplotlib.font_manager - DEBUG - Using fontManager instance from C:\Users\a_h9\.matplotlib\fontlist-v390.json
2024-07-23 19:37:02,899 - matplotlib.pyplot - DEBUG - Loaded backend module://matplotlib_inline.backend_inline version unknown.
2024-07-23 19:37:02,902 - matplotlib.pyplot - DEBUG - Loaded backend inline version unknown.


In [2]:
import pickle
with open('transformed_MNIST/test_data.pkl', 'rb') as f:
  x_test, c_test, y_test = pickle.load(f)

In [3]:
x = torch.tensor(x_test).float()
c = torch.tensor(c_test).float()
y = torch.tensor(y_test).float()

In [4]:
y = F.one_hot(y.long().ravel()).float()

LLR 1

In [8]:
isBias = True

In [17]:
embedding_size = 16
concept_encoder = torch.nn.Sequential(
    torch.nn.Linear(x.shape[1], 32),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(32, 32),
    torch.nn.LeakyReLU(),
    te.nn.ConceptEmbedding(32, c.shape[1], embedding_size),
)

task_predictor = IntpLinearLayer1(embedding_size, y.shape[1], bias=isBias)
model = torch.nn.Sequential(concept_encoder, task_predictor)

In [18]:
import os
model.load_state_dict(torch.load(os.path.join(PATH,'model_state_dict_LLR1_MNIST_Addition_2.pth')))

<All keys matched successfully>

In [19]:
cem = model[0]
llr = model[1]
cem.eval()
llr.eval()
c_emb , c_pred = cem(x)
y_pred, weights, biases = llr(c_emb, c_pred, return_params=True)

In [20]:
y_pred.shape, weights.shape, biases.shape

(torch.Size([10000, 19]), torch.Size([10000, 20, 19]), torch.Size([10000, 19]))

In [36]:
# a function that takes an array where only two elements are non-zero and returns the indices of the non-zero elements
def get_indices(arr):
    return np.where(arr == 1)[0]

c1 = []
c2 = []
for i in range(len(c_test)):
    c1.append(get_indices(c_test[i])[0])
    c2.append(get_indices(c_test[i])[1] - 10)

c1[0], c2[0]

(7, 2)

In [37]:
weights[:,6,3].shape

torch.Size([10000])

In [38]:
df_all = pd.DataFrame()
for i in range(20):
    for j in range(19):
        df_all[f'w{i}_{j}'] = weights[:,i,j].detach().numpy()
df_all['c1'] = c1
df_all['c2'] = c2

df_all.shape

(10000, 382)

In [39]:
df_all[['c1', 'c2', 'w0_10', 'w1_10', 'w2_10', 'w3_10', 'w4_10', 'w5_10', 'w6_10', 'w7_10', 'w8_10', 'w9_10', 'w10_10', 'w11_10', 'w12_10', 'w13_10', 'w14_10', 'w15_10', 'w16_10', 'w17_10', 'w18_10']][df_all['c1']+df_all['c2'] == 10]

,c1,c2,w0_10,w1_10,w2_10,w3_10,w4_10,w5_10,w6_10,w7_10,...,w9_10,w10_10,w11_10,w12_10,w13_10,w14_10,w15_10,w16_10,w17_10,w18_10
1,2,8,2.678248e-06,8.250965e-08,4.594888e-01,1.360929e-03,1.235911e-04,3.987909e-05,0.000029,0.137978,...,4.061863e-04,0.998963,4.056899e-06,3.604923e-05,2.298692e-05,9.532421e-01,0.000025,5.239987e-05,0.015244,9.278960e-01
18,3,7,1.989260e-08,3.462131e-06,3.211980e-06,3.400792e-05,1.349281e-05,1.169693e-05,0.000016,0.924990,...,7.398139e-01,0.999843,2.922779e-02,1.385067e-08,1.098419e-05,1.759046e-03,0.015561,1.629425e-06,0.362497,1.689158e-01
26,7,3,2.062613e-04,9.076451e-14,1.579162e-04,5.759934e-05,5.946417e-08,3.569156e-08,0.000007,0.843119,...,4.089115e-04,0.002144,1.867792e-07,5.749121e-07,9.056249e-01,2.887607e-01,0.810466,8.829854e-01,0.000189,1.749734e-02
32,3,7,8.719292e-09,1.175758e-11,9.561982e-08,9.999393e-01,2.233065e-08,6.813259e-06,0.000012,0.995822,...,3.047712e-03,0.999762,9.785733e-03,1.720083e-10,1.212296e-01,1.790294e-01,0.838811,6.921167e-03,0.935440,9.523737e-04
50,6,4,2.125150e-09,7.487937e-01,3.014007e-05,1.714076e-10,7.068380e-01,1.348082e-04,0.999490,0.092808,...,2.301073e-01,0.979269,2.669715e-04,1.584351e-06,1.191726e-10,9.999937e-01,0.000004,1.418722e-11,0.182727,9.649687e-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9942,3,7,3.238959e-07,1.165694e-14,8.751511e-10,9.999974e-01,4.597678e-10,6.098064e-05,0.000008,0.999450,...,8.189036e-08,0.999983,2.932836e-03,5.519180e-11,8.850334e-01,6.123171e-01,0.996152,4.422181e-01,0.966000,8.357482e-08
9955,1,9,1.010787e-06,1.591645e-03,3.901393e-08,6.974284e-06,3.168689e-06,2.168997e-04,0.000225,0.085691,...,3.671847e-02,0.989920,1.239123e-06,1.202910e-08,2.459974e-10,7.628775e-04,0.018016,2.510816e-06,0.000509,1.354858e-04
9961,6,4,3.928551e-08,3.692721e-01,4.038756e-03,6.856835e-10,3.983028e-01,7.244587e-04,0.999939,0.012108,...,8.277856e-01,0.904181,2.860885e-04,3.400155e-06,2.014009e-10,9.956092e-01,0.000008,1.732142e-11,0.152751,1.276772e-01
9973,9,1,2.377096e-10,1.366662e-05,6.562971e-03,3.135215e-12,2.927953e-04,8.565191e-03,0.000045,0.217981,...,9.985282e-01,0.740113,9.975466e-01,2.588253e-05,4.409461e-09,3.776509e-08,0.006457,1.557004e-06,0.040224,9.916738e-01


LLR 3

In [6]:
isBias = True

In [13]:
embedding_size = 16
concept_encoder = torch.nn.Sequential(
    torch.nn.Linear(x.shape[1], 32),
    torch.nn.LeakyReLU(),
    torch.nn.Linear(32, 32),
    torch.nn.LeakyReLU(),
    te.nn.ConceptEmbedding(32, c.shape[1], embedding_size),
)

task_predictor = IntpLinearLayer3(embedding_size, y.shape[1], bias=isBias)
model = torch.nn.Sequential(concept_encoder, task_predictor)

In [14]:
import os
model.load_state_dict(torch.load(os.path.join(PATH,'model_state_dict_LLR3_MNIST_Addition_2.pth')))

<All keys matched successfully>

In [17]:
cem = model[0]
llr = model[1]
cem.eval()
llr.eval()
c_emb , c_pred = cem(x)
y_pred, weights, biases = llr(c_emb, c_pred, return_params=True)

In [18]:
y_pred.shape, weights.shape, biases.shape

(torch.Size([10000, 19]), torch.Size([10000, 20, 19]), torch.Size([10000, 19]))

In [20]:
# a function that takes an array where only two elements are non-zero and returns the indices of the non-zero elements
def get_indices(arr):
    return np.where(arr == 1)[0]

c1 = []
c2 = []
for i in range(len(c_test)):
    c1.append(get_indices(c_test[i])[0])
    c2.append(get_indices(c_test[i])[1] - 10)

c1[0], c2[0]

(7, 2)

In [21]:
df_all = pd.DataFrame()
for i in range(20):
    for j in range(19):
        df_all[f'w{i}_{j}'] = weights[:,i,j].detach().numpy()
df_all['c1'] = c1
df_all['c2'] = c2

df_all.shape

(10000, 382)

In [22]:
df_all[['c1', 'c2', 'w0_10', 'w1_10', 'w2_10', 'w3_10', 'w4_10', 'w5_10', 'w6_10', 'w7_10', 'w8_10', 'w9_10', 'w10_10', 'w11_10', 'w12_10', 'w13_10', 'w14_10', 'w15_10', 'w16_10', 'w17_10', 'w18_10']][df_all['c1']+df_all['c2'] == 10]

,c1,c2,w0_10,w1_10,w2_10,w3_10,w4_10,w5_10,w6_10,w7_10,...,w9_10,w10_10,w11_10,w12_10,w13_10,w14_10,w15_10,w16_10,w17_10,w18_10
1,2,8,-0.908285,0.990836,0.972473,0.827701,0.251889,-0.938443,-1.000000,-0.123302,...,0.112292,-0.713936,0.921726,0.999250,0.326748,-0.997101,-0.999794,-1.000000,0.005214,0.337785
18,3,7,0.049160,0.618103,-0.243890,-0.000920,0.758038,-0.464929,-0.999997,0.020944,...,-0.016144,-0.354268,0.980167,0.982778,0.840036,-0.950650,-0.999964,-1.000000,-0.667816,-0.850853
26,7,3,0.280325,0.312136,-0.994460,-0.601355,-0.978601,0.469493,-0.999271,0.602589,...,-0.187212,-0.995122,0.994277,-0.998200,0.652756,-0.996215,-0.848452,-0.999993,0.850153,-0.987771
32,3,7,0.120497,0.969062,0.112608,0.966874,-0.787435,0.463597,-1.000000,0.827320,...,-0.042294,-0.888301,0.990137,0.999979,0.964545,-0.999966,-0.999971,-1.000000,0.836035,-0.553365
50,6,4,-0.039497,0.836067,-0.704872,0.855649,0.999156,-0.999925,0.790762,-0.190064,...,0.514045,-0.031809,0.989535,-0.999999,0.679989,0.999177,-1.000000,-1.000000,-0.999996,-0.999563
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9942,3,7,-0.078108,0.982820,0.313464,0.982003,-0.990395,-0.109883,-1.000000,0.966822,...,-0.127780,-0.973439,0.634731,0.999981,0.983756,-1.000000,-0.999946,-1.000000,0.991558,-0.693807
9955,1,9,0.049520,0.271270,-0.160489,0.835162,0.732599,-0.951029,-0.999985,-0.057488,...,-0.001612,-0.569279,0.993711,-0.714317,-0.865090,0.126620,-0.997090,-0.999983,-0.022767,-0.997897
9961,6,4,0.056340,0.573915,-0.881809,0.752456,0.998209,-0.999967,0.422636,-0.794722,...,0.155125,-0.065891,0.801955,-1.000000,-0.660150,0.989380,-1.000000,-0.999999,-0.999991,-0.999854
9973,9,1,0.571188,-0.980406,-0.999202,0.658545,0.998429,-0.999450,-0.651514,-0.999992,...,0.983602,-0.872696,0.977504,-0.999637,-0.374952,-0.040732,-0.999935,-0.999949,-0.999972,-1.000000
